In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

# Machine Learning: Text Classification Assignment

In [2]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import train_test_split as tts
from nltk.corpus.reader.plaintext import CategorizedPlaintextCorpusReader

import requests
from bs4 import BeautifulSoup

<IPython.core.display.Javascript object>

### Use the CategorizedPlaintextCorpusReader to import the AP_News corpus.

In [3]:
PATH = "corpata/AP_News"

DOC_PATTERN = r".*\.txt"
CAT_PATTERN = r"([\w_\s]+)"

corpus = CategorizedPlaintextCorpusReader(PATH, DOC_PATTERN, cat_pattern=CAT_PATTERN)

<IPython.core.display.Javascript object>

### Create two separate lists - one containing the text from each document and another containing the category of each article in the corpus.

In [4]:
docs = [corpus.raw(fileid) for fileid in corpus.fileids()]

# categories = [corpus.categories(fileid)[0] for fileid in corpus.fileids()]
categories = [fileid.split("/")[0] for fileid in corpus.fileids()]

<IPython.core.display.Javascript object>

### Preprocess the corpus, ensuring to include the following steps.

- Word tokenize the documents.
- Lemmatize, stem, and lowercase all tokens.
- Remove punctuation and stop words.

In [5]:
def preprocess(docs):
    lemmatizer = WordNetLemmatizer()
    stemmer = SnowballStemmer("english")
    preprocessed = []

    for doc in docs:
        tokenized = word_tokenize(doc)

        cleaned = [
            stemmer.stem(lemmatizer.lemmatize(token.lower()))
            for token in tokenized
            if not token.lower() in stopwords.words("english")
            if token.isalpha()
        ]
        untokenized = " ".join(cleaned)
        preprocessed.append(untokenized)

    return preprocessed

<IPython.core.display.Javascript object>

In [6]:
preprocessed = preprocess(docs)

<IPython.core.display.Javascript object>

In [7]:
preprocessed[0]

'honolulu ap univers hawaii seek addit fund student mental health servic scholarship item new supplementari budget request offici said board regent approv fiscal year supplement oper budget million thursday honolulu report request submit state legislatur democrat gov univers request million hire psychologist system univers hawaii manoa eight psychologist hilo campus three west oahu campus posit communiti colleg one posit said allyson tanouy coordin mental health throughout univers system nation standard one mental health profession per student tanouy said add posit one per low mental health fund would also expand program prevent suicid reduc mental health stigma provid peer educ alert new student parent colleg transit challeng offici said largest item supplement budget million expand hawaii promis program scholarship state institut univers propos flat amount cover tuition fee hawaii resid qualifi feder pell grant look focus needi student go campus said donald straney vice presid academ

<IPython.core.display.Javascript object>

### Split the data into training and testing sets with the size of the test set being 30% of the records.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    preprocessed, categories, test_size=0.3, random_state=42
)

<IPython.core.display.Javascript object>

### Construct a pipeline that TF-IDF vectorizes the text and trains a Random Forest classification model.

In [9]:
model = Pipeline(
    [
        #         ("vect", CountVectorizer()),
        #         ("tfidf", TfidfTransformer()),
        ("tfidf", TfidfVectorizer()),
        ("rfc", RandomForestClassifier()),
    ]
)

model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None

<IPython.core.display.Javascript object>

In [10]:
model.score(X_test, y_test)

0.8484848484848485

<IPython.core.display.Javascript object>

### Generate predictions on the test set and print a classification report to evaluate how well the model performed.

In [11]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      health       0.69      0.92      0.79        12
    politics       0.90      0.90      0.90        20
      sports       0.95      0.95      0.95        20
        tech       0.80      0.57      0.67        14

    accuracy                           0.85        66
   macro avg       0.83      0.83      0.83        66
weighted avg       0.86      0.85      0.84        66



<IPython.core.display.Javascript object>

### Perform 10-fold cross validation and obtain the averge F1 score across all the folds.

In [12]:
scores = cross_val_score(model, preprocessed, categories, cv=10, scoring="f1_macro")

scores.mean()

0.807902097902098

<IPython.core.display.Javascript object>

### Ingest, preprocess, and predict the topic of the article at the following URL.

In [13]:
url = "https://www.nytimes.com/2019/11/25/business/uber-london.html"

response = requests.get(url)
content = response.text

soup = BeautifulSoup(content)

cleaned_lines = [
    line.text for line in soup.find_all("p", class_="css-158dogj evys1bk0")
]

<IPython.core.display.Javascript object>

In [14]:
doc = " ".join(cleaned_lines)
processed = preprocess([doc])

model.predict(processed)

array(['tech'], dtype='<U8')

<IPython.core.display.Javascript object>